In [335]:
import pandas as pd
import numpy as np

In [336]:
# Most seen delegates again, but they don't have to be officially listed as the delegate of the comp
results_path = 'WCA_export_Results.tsv'
results_df = pd.read_table(results_path)[['competitionId', 'personId']]
results_df.head()

,competitionId,personId
0,LyonOpen2007,2007AMAN01
1,LyonOpen2007,2004ROUA01
2,LyonOpen2007,2005SIMO01
3,LyonOpen2007,2007MALL01
4,LyonOpen2007,2007DESM01


In [337]:
comp_path = 'WCA_export_Competitions.tsv'
comp_df = pd.read_table(comp_path)
fmc_only_comps = comp_df[comp_df['eventSpecs'] == '333fm'][['id']]
fmc_only_comp_ids = fmc_only_comps['id'].values

regular_comps = comp_df[~comp_df['id'].isin(fmc_only_comp_ids)][['id', 'wcaDelegate']]
regular_comp_results = results_df[~results_df['competitionId'].isin(fmc_only_comp_ids)]

In [338]:
regular_comp_results = regular_comp_results.drop_duplicates()

In [339]:
def split_delegates(x):
    delegates = x.split('] ')
    delegate_names = []
    for d in delegates:
        delegate = d[2:d.index('}')]
        delegate_names.append(delegate)
    return delegate_names
regular_comps['delegates'] = regular_comps['wcaDelegate'].apply(split_delegates)
regular_comps = regular_comps.drop(columns=['wcaDelegate'])

In [340]:
# To find delegates who mapped to multiple ids (see below cell)
# for d, i in zip(regular_comps['delegates'], regular_comps['id']):
#     if 'Ray Li' in d:
#         print('found')
#         print(i)

In [341]:
delegate_names = list(set(regular_comps['delegates'].agg(sum)))
delegate_names = pd.DataFrame(delegate_names)
delegate_names = delegate_names.rename(columns={0:'name'})

persons_path = 'WCA_export_Persons.tsv'
persons_df = pd.read_table(persons_path)[['id', 'name']]

In [342]:
delegates_df = delegate_names.merge(persons_df, how='left', left_on='name', right_on='name')
delegates_df = delegates_df.drop_duplicates() # Reto Bubendorf appeared twice in the merge for some reason...
# delegates_df[delegates_df['id'].isnull()] # Delegates who have never competed like Erno Rubik
delegates_df = delegates_df.drop(axis=0, labels=[25,50,89,248]) # Drop the above delegates


In [343]:
# Fix delegates who when merged, mapped to multiple ids
delegates_to_fix = delegates_df[['name']].groupby('name').filter(lambda x: len(x) > 1).drop_duplicates()
delegates_to_fix['id'] = 0
delegates_to_fix.loc[delegates_to_fix['name'] == 'Gábor Szabó', 'id'] = '2005SZAB02' # Delegated at GalantaOpen2009
delegates_to_fix.loc[delegates_to_fix['name'] == 'Chenxi Shan (单晨曦)', 'id'] = '2010SHAN02'
delegates_to_fix.loc[delegates_to_fix['name'] == 'Zachary White', 'id'] = '2010WHIT05'
delegates_to_fix.loc[delegates_to_fix['name'] == 'Ayush Kumar', 'id'] = '2011KULS01'
delegates_to_fix.loc[delegates_to_fix['name'] == 'Zheng Li (李政)', 'id'] = '2008LIZH02'
delegates_to_fix.loc[delegates_to_fix['name'] == 'Artem Melikian (Артем Мелікян)', 'id'] = '2011MELI01'
delegates_to_fix.loc[delegates_to_fix['name'] == 'Evan Liu', 'id'] = '2009LIUE01'
delegates_to_fix.loc[delegates_to_fix['name'] == 'Caleb Hoover', 'id'] = '2016HOOV01'
delegates_to_fix.loc[delegates_to_fix['name'] == 'Jason White', 'id'] = '2016WHIT16'
delegates_to_fix.loc[delegates_to_fix['name'] == 'Felix Lee', 'id'] = '2008LEEF01'
delegates_to_fix.loc[delegates_to_fix['name'] == 'Ray Li', 'id'] = '2010LIRA01'
delegates_to_fix.loc[delegates_to_fix['name'] == 'Adam Walker', 'id'] = '2012WALK02'
delegates_to_fix_names = delegates_to_fix['name']

In [344]:
delegates_df = delegates_df.drop(delegates_df[delegates_df.name.isin(delegates_to_fix_names)].index)

In [345]:
final_delegates = pd.concat([delegates_df, delegates_to_fix])
final_delegate_ids = list(final_delegates['id'])

In [346]:
def intersect(a, b):
    # Thanks https://www.saltycrane.com/blog/2008/01/how-to-find-intersection-and-union-of/
    """ return the intersection of two lists """
    return list(set(a) & set(b))

def find_delegates(x):
    # Per competition, find the delegates that COMPETED there
    competitors = x['personId']
    return intersect(competitors, final_delegate_ids)
    
comp_to_delegates = regular_comp_results.groupby('competitionId').apply(find_delegates).to_frame().reset_index()
comp_to_delegates = comp_to_delegates.rename(columns={0:"delegateIds"})
comp_to_delegates.head()

,competitionId,delegateIds
0,100Merito2018,"[2014SANT16, 2007CINO01]"
1,12SidesofSilesia2018,[2012TRZA01]
2,150thCubeMeetinginBrest2017,"[2012TERE01, 2014PYAT01]"
3,1AVG2013,"[2003BRUC01, 2010BICL01, 2003VAND01, 2006BUUS01]"
4,1BodyCubing2017,"[2009BLAI01, 2005HAYE01]"


In [351]:
person_to_delegates = regular_comp_results.merge(comp_to_delegates, how='inner', left_on='competitionId', right_on='competitionId')
person_to_delegates = person_to_delegates[['personId', 'delegateIds']]
person_to_delegates = person_to_delegates.groupby('personId').agg(sum)

In [352]:
person_to_delegates['delegateIds'] = person_to_delegates['delegateIds'].apply(lambda x: set(x))
person_to_delegates = person_to_delegates.reset_index()

In [353]:
person_to_delegates['numDelegates'] = person_to_delegates['delegateIds'].apply(lambda x: len(x))

In [355]:
final_df = person_to_delegates.sort_values(by='numDelegates', ascending=False)

In [360]:
final_df.merge(persons_df, how='inner', left_on='personId', right_on='id')[['name', 'id', 'numDelegates', 'delegateIds']]


,name,id,numDelegates,delegateIds
0,Mats Valk,2007VALK01,240,"{2003BRUC01, 2009LIUE01, 2009JARU02, 2013BULM0..."
1,Ron van Bruchem,2003BRUC01,230,"{2003BRUC01, 2009LIUE01, 2009JARU02, 2007CINO0..."
2,Shelley Chang,2004CHAN04,224,"{2003BRUC01, 2009LIUE01, 2009JARU02, 2013BULM0..."
3,Jeremy Fleischman,2005FLEI01,223,"{2003BRUC01, 2009LIUE01, 2009JARU02, 2013BULM0..."
4,Feliks Zemdegs,2009ZEMD01,222,"{2003BRUC01, 2009LIUE01, 2009JARU02, 2013BULM0..."
5,Hanneke Rijks,2008RIJK01,219,"{2003BRUC01, 2009LIUE01, 2009JARU02, 2007CINO0..."
6,Jasmine Lee,2003LEEJ01,217,"{2003BRUC01, 2009LIUE01, 2009JARU02, 2013BULM0..."
7,Tim Reynolds,2005REYN01,216,"{2003BRUC01, 2009LIUE01, 2009JARU02, 2013BULM0..."
8,Chris Hardwick,2003HARD01,213,"{2003BRUC01, 2009LIUE01, 2009JARU02, 2007CINO0..."
9,Sinpei Araki (荒木慎平),2006ARAK01,212,"{2003BRUC01, 2009LIUE01, 2009JARU02, 2013BULM0..."
